In [1]:
import numpy as np
import os
import warnings

import torch

warnings.filterwarnings("ignore")

from data_loader import load_mnist_data
from supervised_models_mlp import MLP, train_mlp_pytorch, predict_mlp_pytorch
from supervised_models_xgboost import xgb_model
from supervised_models_logit import logit

from deae_self import VIME_Self
from deae_semi import train_model
from deae_utils import perf_metric

# Experimental parameters
label_no = 1000
model_sets = []

# Hyper-parameters
p_m = 0.3
alpha = 2
K = 3
beta = 1.0
label_data_rate = 0.3

# Metric
metric = 'acc'

# Define output
results = np.zeros([len(model_sets) + 2])

x_train, y_train, x_unlab, x_test, y_test = load_mnist_data(label_data_rate)

# Train VIME-Self
vime_self_parameters = dict()
vime_self_parameters['batch_size'] = 128
vime_self_parameters['epochs'] = 1000

file_name = './save_model/encoder_model'

_, dim = x_unlab.shape
vime_self_model = VIME_Self(dim, alpha)
vime_self_model.train_model(x_unlab, p_m, vime_self_parameters)

# Save encoder
if not os.path.exists('save_model'):
    os.makedirs('save_model')
# 假设 model 是你的 VIME_Self 模型实例
torch.save(vime_self_model.encoder.state_dict(), file_name)

# 首先创建一个与保存的编码器结构相同的新编码器实例
encoder = VIME_Self(dim, alpha).encoder  # 假设 dim 和 alpha 已经定义

# 加载保存的 state_dict
encoder.load_state_dict(torch.load(file_name))
encoder.eval()  # 切换到评估模式

# MLP
input_dim = x_train.shape[1]
hidden_dim = 64  # 示例隐藏层维度
output_dim = y_train.shape[1] # 假设y_train是类别索引
activation_fn = 'relu'

model = MLP(input_dim, hidden_dim, output_dim, activation_fn)

mlp_parameters = {
    'batch_size': 128,
    'epochs': 100,
    'lr': 0.01,
}

x_train_tensor = torch.tensor(x_train, dtype=torch.float32)
x_test_tensor = torch.tensor(x_test, dtype=torch.float32)

x_train_hat = encoder(x_train_tensor)
x_test_hat = encoder(x_test_tensor)

train_mlp_pytorch(x_train_hat, y_train, model, mlp_parameters)
y_test_hat = predict_mlp_pytorch(x_test_hat, model)

# train_mlp_pytorch(x_train, y_train, model, mlp_parameters)
# y_test_hat = predict_mlp_pytorch(x_test, model)

results[0] = perf_metric(metric, y_test, y_test_hat)

print('VIME-Self Performance: ' + str(results[0]))

Epoch 1/1000, Total Loss: 18.2333, Mask Loss: 11.3263, Feature Loss: 3.4535
Epoch 2/1000, Total Loss: 15.1613, Mask Loss: 11.0381, Feature Loss: 2.0616
Epoch 3/1000, Total Loss: 12.4411, Mask Loss: 10.9127, Feature Loss: 0.7642
Epoch 4/1000, Total Loss: 11.3740, Mask Loss: 10.9227, Feature Loss: 0.2256
Epoch 5/1000, Total Loss: 11.2169, Mask Loss: 11.0181, Feature Loss: 0.0994
Epoch 6/1000, Total Loss: 11.2018, Mask Loss: 11.0721, Feature Loss: 0.0649
Epoch 7/1000, Total Loss: 11.1344, Mask Loss: 11.0325, Feature Loss: 0.0509
Epoch 8/1000, Total Loss: 11.1959, Mask Loss: 11.1088, Feature Loss: 0.0435
Epoch 9/1000, Total Loss: 11.0368, Mask Loss: 10.9589, Feature Loss: 0.0389
Epoch 10/1000, Total Loss: 11.1200, Mask Loss: 11.0478, Feature Loss: 0.0361
Epoch 11/1000, Total Loss: 10.9527, Mask Loss: 10.8853, Feature Loss: 0.0337
Epoch 12/1000, Total Loss: 10.9505, Mask Loss: 10.8860, Feature Loss: 0.0323
Epoch 13/1000, Total Loss: 11.0503, Mask Loss: 10.9886, Feature Loss: 0.0308
Epoch 14

In [4]:
lr = 0.01

# Train VIME-Semi
vime_semi_parameters = dict()
vime_semi_parameters['hidden_dim'] = 32
vime_semi_parameters['batch_size'] = 128
vime_semi_parameters['iterations'] = 201
y_test_hat = train_model(encoder, x_train, y_train, x_unlab, x_test,
                       vime_semi_parameters, p_m, K, beta, lr)

# Test VIME
results[1] = perf_metric(metric, y_test, y_test_hat)

print('VIME Performance: ' + str(results[1]))

Iteration: 0/201, Current Total Loss: 5.239163909639631
Iteration: 100/201, Current Total Loss: 4.03833362034389
Iteration: 200/201, Current Total Loss: 3.990338615008763
VIME Performance: 0.47724477244772445
